To use base conda environment to run scraping codes

### Scraping codes to scrape glassdoor's DBS reviews (Sort to show the most recent reviews)

Countries of interest: Singapore, Taiwan, Hong Kong, India, China, Indonesia

Positions of interest: Contract, Part-time, Full-time

To select companies based on P&L 

Group 1: Foreign banks in Singapore: Source (https://sgbanks.com)
(JP Morgan Singapore, Goldman Sachs)
 
Group 2: Local Banks (UOB, OCBC)
- UOB ratings - 3.5
- OCBC ratings - 3.6

Group 3: Source (https://www.glassdoor.com/Explore/top-information-technology-companies-singapore_IS.4,26_ISEC10013_IL.37,46_IM1123.htm) 
Tech Companies (Google, Apple, Microsoft)

### Steps to be automated:
1. On page 1, clicking on "Continue Reading" for each review will cause a pop up to appear
2. Click on 'x' for pop up
3. After scraping all info for 1st page, move on to next page
4. A 'sign up' screen will then appear
5. Click on blue 'sign in' button
6. Input email and password into 'sign in' fields and click on blue 'sign in' button
7. After step 6, will land on pg 2
8. Proceed to click on 'Continue Reading' for all reviews. But this time, no pop up will appear
9. Scrape wanted info and move on to next page
10. Repeat step 9 for the rest of the pages

*Dataframe to be saved as xlsx as saving in csv may have some problem*

Note when running the following codes:

- If you face **WebdriverException** or **ElementClickException**, restart kernel and rerun `scraping_glassdoor_v1()`

    - **I recommend scraping only 4000 reviews at one go**. E.g., if you want to scrape reviews of a company with 10k reviews (1000 pages), I recommend scraping 4000, 4000, 2000 reviews.

    - If you do that, then you need to place the url for the 1st page, 401st page and 801st page into the 2nd parameter of scraping_glassdoor_v1()

- You need to install chromedriver before using this code

- Replace the 3rd and 4th parameter of scraping_glassdoor_v1() with your email and glassdoor password

In [6]:
# Codes to scrape Glassdoor for reviews like DBS - those that do not produce any exception
import time
import math
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

def scraping_glassdoor_v1(num_reviews:int, url:str, email:str, password:str):
    
    '''
    num_reviews: Number of reviews to be retrieved at one go
    url: Input url that reviews should be scraped from (vary according to country of interest)
    email: Input email that is registered on glassdoor
    password: Input password 
    '''

    date_occupation=[]
    rating=[]
    pros_list=[]
    cons_list=[]
    country_list=[]
    page_list=[]
    current_or_former_employee=[]
    recommended=[]
    ceoApproval=[]
    business_Outlook=[]
    

    driver = webdriver.Chrome(r'C:\Webdriver\chromedriver') # put directory that chromedriver is stored in
    driver.get(url)
    
    for i in range(math.ceil(num_reviews/10)):
        
        intermediate_page_list=[]

        if EC.visibility_of_all_elements_located((By.XPATH, "//*[@class='v2__EIReviewDetailsV2__continueReading']")):
            time.sleep(5)
            if i == 0: # Do the following steps if it is the first page

                # If there are "continue reading" buttons, click on those to show full reviews
                for review in driver.find_elements_by_class_name("v2__EIReviewDetailsV2__continueReading"):
                    review.click()

                    # If prompt appears, click x to close it 
                    try:
                        driver.find_element_by_class_name("modal_closeIcon").click()  #clicking on the X.
                        driver.find_element_by_class_name("modal_closeIcon-svg").click()  #clicking on the X.
                    except NoSuchElementException:
                        pass
            else:
                for review in driver.find_elements_by_class_name("v2__EIReviewDetailsV2__continueReading"):
                    driver.execute_script("var x= document.getElementsByClassName('v2__EIReviewDetailsV2__continueReading')[0];"+"x.click();")

            source = driver.page_source
            soup = BeautifulSoup(source,'lxml')
            
            # To obtain recommended/ ceo approval/ business outlook
            for div in soup.find_all('div', attrs={'class':'mt-xxsm'}):
                intermediate=[]
                j=0
                for div2 in div.div.div.div.div:
                    j+=1
                    for div3 in div2:
                        try:
                            if j % 3 == 1:
                                if str(div3.svg.get('class')[1]) == 'css-10xv9lv-svg':
                                    recommended.append("Neutral")
                                elif str(div3.svg.get('class')[1]) == "css-hcqxoa-svg":
                                    recommended.append("Recommended")
                                else:
                                    recommended.append("Not Recommended")
                            elif j % 3 == 2:
                                if str(div3.svg.get('class')[1]) == 'css-10xv9lv-svg' or str(div3.svg.get('class')[1]) == 'css-1h93d4v-svg':
                                    ceoApproval.append("Neutral")
                                elif str(div3.svg.get('class')[1]) == "css-hcqxoa-svg":
                                    ceoApproval.append("Recommended")
                                else:
                                    ceoApproval.append("Not Recommended")
                            else:
                                if str(div3.svg.get('class')[1]) == 'css-10xv9lv-svg' or str(div3.svg.get('class')[1]) == 'css-1h93d4v-svg':
                                    business_Outlook.append("Neutral")
                                elif str(div3.svg.get('class')[1]) == "css-hcqxoa-svg":
                                    business_Outlook.append("Recommended")
                                else:
                                    business_Outlook.append("Not Recommended")
                        except:
                            pass
            
            for date_occ in soup.find_all('span', attrs={'class':'authorJobTitle middle common__EiReviewDetailsStyle__newGrey'}):
                date_occupation.append(date_occ.text)

            # Add all ratings into ratings list
            for ratings in soup.find_all('span', attrs={'class':'ratingNumber mr-xsm'}):
                rating.append(ratings.text)
            
            # Get employee type (current/ former employee)
            for employee_type in soup.find_all('span', attrs={'class':'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'}):
                current_or_former_employee.append(employee_type)

            # Add all pros into pros_list
            for pros in soup.find_all('span', attrs={'data-test':'pros'}):
                pros_list.append(pros.text)

            # Add all cons into cons list
            for cons in soup.find_all('span', attrs={'data-test':'cons'}):
                cons_list.append(cons.text)
            
            # To go to next page
            WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class='nextButton css-1hq9k8 e13qs2071']")))
            driver.execute_script("var x= document.getElementsByClassName('nextButton css-1hq9k8 e13qs2071')[0];"+"x.click();")

            
            if i == 0: # Sign in pop up will only appear for page 1
                if EC.visibility_of_all_elements_located((By.XPATH, "//*[@class='link ml-xxsm']")):

                    # Click on Sign in button (after page 1) to get from sign up to sign in page
                    WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class='link ml-xxsm']")))
                    driver.execute_script("var x= document.getElementsByClassName('link ml-xxsm')[0];"+"x.click();")

                    email_field = driver.find_element(By.ID,'hardsellUserEmail')
                    password_field = driver.find_element(By.ID,'hardsellUserPassword')

                    email_field.send_keys(email)
                    password_field.send_keys(password)

                    # Click sign in button after inputting login credentials
                    WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class='gd-ui-button mt-std minWidthBtn css-14xfqow evpplnh0']")))
                    driver.execute_script("var x= document.getElementsByClassName('gd-ui-button mt-std minWidthBtn css-14xfqow evpplnh0')[0];"+"x.click();")
                    
    country_list.append(soup.find_all('h1', attrs={'class':'eiReviews__EIReviewsPageStyles__newPageHeader'})[0])
    country_list = country_list * num_reviews
    
    return date_occupation, rating, pros_list, cons_list, country_list, page_list, current_or_former_employee, recommended, ceoApproval, business_Outlook

# https://www.glassdoor.sg/Reviews/Apple-Reviews-E1138_P2.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=REGULAR&filter.employmentStatus=PART_TIME&filter.employmentStatus=CONTRACT
date_occupation, rating, pros_list, cons_list, country_list, page_list, current_or_former_employee, recommended, ceoApproval, business_Outlook = scraping_glassdoor_v1(4000, 'https://www.glassdoor.sg/Reviews/Apple-Reviews-E1138_P2801.htm?sort.sortType=RD&sort.ascending=false&filter.iso3Language=eng&filter.employmentStatus=REGULAR&filter.employmentStatus=PART_TIME&filter.employmentStatus=CONTRACT',\
                                                                                                                                                                    "email",
                                                                                                                                                                    "glassdoor_password")

C:\Users\jingh\AppData\Local\Temp/ipykernel_9020/544448734.py:36: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Webdriver\chromedriver')
C:\Users\jingh\AppData\Local\Temp/ipykernel_9020/544448734.py:48: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  for review in driver.find_elements_by_class_name("v2__EIReviewDetailsV2__continueReading"):
C:\Users\jingh\AppData\Local\Temp/ipykernel_9020/544448734.py:53: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("modal_closeIcon").click()  #clicking on the X.
C:\Users\jingh\AppData\Local\Temp/ipykernel_9020/544448734.py:54: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("

In [7]:
date_occupation_df = pd.DataFrame(date_occupation, columns = ['Date_occupation'])
date_occupation_df[['Date', 'Occupation']]=date_occupation_df['Date_occupation'].str.split("-", n=1, expand=True)
date_occupation_df=date_occupation_df.drop(['Date_occupation'],axis=1)

pros_df = pd.DataFrame(pros_list, columns = ['Pros'])
cons_df = pd.DataFrame(cons_list, columns = ['Cons'])
rating_df = pd.DataFrame(rating, columns = ['Rating'])
country_df = pd.DataFrame(country_list, columns = ['Review_type'])
current_or_former_df = pd.DataFrame(current_or_former_employee, columns = ['Employee Type'])
recommended_df = pd.DataFrame(recommended, columns = ['Recommended Or Not'])
ceoApproval_df = pd.DataFrame(ceoApproval, columns = ['CEO Approval'])
business_Outlook_df = pd.DataFrame(business_Outlook, columns = ['Business Outlook'])

full_df = pd.concat([country_df, date_occupation_df, current_or_former_df, pros_df, cons_df, recommended_df, ceoApproval_df, business_Outlook_df, rating_df], axis=1)

full_df.to_excel("..\Employee Reviews\Apple Reviews\All Apple Reviews 8.xlsx", index=False)
# full_df
full_df.drop_duplicates()

,Review_type,Date,Occupation,Employee Type,Pros,Cons,Recommended Or Not,CEO Approval,Business Outlook,Rating
0,Apple Reviews,18 Dec 2011,Specialist,Current Employee,"Great people, Great Product, Extremely friendl...",Not enough staff members to help everyone. Cus...,Recommended,Recommended,Neutral,4.0
1,Apple Reviews,18 Dec 2011,,Current Employee,- You work with awesome people\r\n- Salary and...,"- retail is retail, working hours sucks\r\n- o...",Neutral,Recommended,Neutral,3.0
2,Apple Reviews,15 Dec 2011,Sales Specialist,Current Employee,Working with technology every day.,"It is still retail, lousy pay.",Recommended,Recommended,Neutral,4.0
3,Apple Reviews,15 Dec 2011,Mac Genius,Current Employee,"Decent discounts, good health, vision, dental....",Can tend to be over worked. Breaks are not giv...,Not Recommended,Neutral,Neutral,1.0
4,Apple Reviews,15 Dec 2011,,Current Employee,"Apple provides creative freedom, great pay, gr...",up to this point I have no complaints,Recommended,Recommended,Neutral,5.0
...,...,...,...,...,...,...,...,...,...,...
115,NaN,11 Oct 2011,Senior Software Engineer,Current Employee,Engineers at Apple get to make real contributi...,Products go out the door on a schedule. Those...,Recommended,Recommended,Neutral,5.0
116,NaN,10 Oct 2011,Manager,Current Employee,"Shares, but now they have peaked its not worth...","The senior management are inept, have not kept...",Not Recommended,Neutral,Neutral,1.0
117,NaN,9 Oct 2011,Program Manager,Current Employee,Apple has great teams. Folks respect each othe...,It is very demanding env at Apple. Folks works...,Neutral,Recommended,Neutral,4.0
118,NaN,9 Oct 2011,,Current Employee,Atmosphere. Awesome coworkers who are knowledg...,"Constantly busy, which can be looked as a pro ...",Recommended,Recommended,Neutral,4.0


In [9]:
import pandas as pd
intermediate1 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 1.xlsx')
intermediate2 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 2.xlsx')
intermediate3 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 3.xlsx')
intermediate4 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 4.xlsx')
intermediate5 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 5.xlsx')
intermediate6 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 6.xlsx')
intermediate7 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 7.xlsx')
intermediate8 = pd.read_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All Apple Reviews 8.xlsx')

full_df = pd.concat([intermediate1, intermediate2, intermediate3, intermediate4, intermediate5, intermediate6, intermediate7, intermediate8],axis=0)
full_df = full_df[~pd.isna(full_df['Date'])] 
full_df.drop_duplicates().reset_index().drop(['index'], axis=1)#s.to_excel(r'C:\Users\jingh\OneDrive\Desktop\DBS Internship stuff\Employee Reviews Documents\Employee Reviews\Apple Reviews\All_Apple_Reviews.xlsx', index=False)
# intermediate.drop_duplicates()